# Read me before using

This was my attempt at applying neural network modeling to predicting electron total energy flux from LBHL and LBHS emission data using only 1 week of data. Because of the small amount of data, I omitted k cross fold validation and a bunch of necessary steps that would make this a big data problem. 

In [1]:
import numpy as np
import h5py 
import os
from sklearn.model_selection import train_test_split 
import matplotlib.pyplot as plt 
from sklearn import svm
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from collections import defaultdict
from PIL import Image
from tensorflow import keras
from sklearn.preprocessing import StandardScaler, PowerTransformer, RobustScaler
import glob
import pdb
from scipy.ndimage import uniform_filter1d
import pickle


# Get the Conjunction data

In [2]:
hd_dir = os.path.join('/home/matsuo/amgeo_dev/Pseudo_ML','conjunction_data2')
file_dir = os.path.join(hd_dir,'*.hdf5')
conjunc_files = glob.glob(file_dir)
conjunc_files = np.sort(conjunc_files) #sort the files by time 

ele_diff_energy_flux_arr,ion_diff_energy_flux_arr = np.empty((0,19)) ,np.empty((0,19))
ele_flux, ion_flux = [],[]
ssusi_lbhl, ssusi_lbhs = [],[]
jds, lons, lats = [], [], []
sat_nums, passes = [], []

ssj_lbhl, ssj_lbhs = [],[]

for file_name in conjunc_files:
    with h5py.File(file_name, 'r') as f:

        jds.extend(f['jds'][:])
        passes.extend(f['pass_num'][:])
        sat_nums.extend(f['sat_no'][:])
        lons.extend(f['lons'][:])
        lats.extend(f['lats'][:])
        
        #input data
        ssusi_lbhl.extend(f['LBHL'][:])
        ssusi_lbhs.extend(f['LBHS'][:])
        #output 
        ele_diff_energy_flux_arr = np.vstack((ele_diff_energy_flux_arr, f['ele_diff_energy_flux'][:]))
        ion_diff_energy_flux_arr = np.vstack((ion_diff_energy_flux_arr, f['ion_diff_energy_flux'][:]))
        ele_flux.extend(f['ele_total_energy_flux'][:])
        ion_flux.extend(f['ion_total_energy_flux'][:])

        

In [3]:
#convert to numpy arrays 
ele_flux, ion_flux = np.asarray(ele_flux), np.asarray(ion_flux)
ssusi_lbhl, ssusi_lbhs = np.asarray(ssusi_lbhl), np.asarray(ssusi_lbhs)
jds = np.asarray(jds)
sat_nums, passes = np.asarray(sat_nums), np.asarray(passes)
lons, lats = np.asarray(lons), np.asarray(lats)
ssj_lbhl, ssj_lbhs = np.asarray(ssj_lbhl), np.asarray(ssj_lbhs)

## Feature Engineering

### Format the data and get rid of invalid data

In [4]:
ssusi_lbhl_mask = np.isfinite(ssusi_lbhl)
ssusi_lbhs_mask = np.isfinite(ssusi_lbhs)
ele_flux_mask = np.isfinite(ele_flux)
ion_flux_mask = np.isfinite(ion_flux)

mask_big = np.logical_and.reduce([ssusi_lbhl_mask, ssusi_lbhs_mask, ele_flux_mask,ion_flux_mask]).flatten()

In [5]:
#apply the masks
ele_diff_energy_flux_arr = ele_diff_energy_flux_arr[mask_big]
lons, lats = lons[mask_big], lats[mask_big]
ssj_lbhl, ssj_lbhs = ssj_lbhl[mask_big], ssj_lbhs[mask_big]
ele_flux, ion_flux, ssusi_lbhl, ssusi_lbhs, jds, sat_nums, passes = ele_flux[mask_big], ion_flux[mask_big], ssusi_lbhl[mask_big], ssusi_lbhs[mask_big], jds[mask_big], sat_nums[mask_big], passes[mask_big]

### Smooth ion_flux and ele_flux isolate signal

In [6]:
#iterate across the passs
unique_passes = np.unique(passes)

pixel_smooth = 5 
ele_flux_smoothed, ion_flux_smoothed = np.zeros_like(ele_flux), np.zeros_like(ion_flux)
for unique_pass in unique_passes:
    #get the observations in the pass
    mask = unique_pass == passes
    ele_flux_in_pass, ion_flux_in_pass = ele_flux[mask], ion_flux[mask]
    #smooth the eflux observations in the pass
    ele_flux_smoothed[mask], ion_flux_smoothed[mask] = uniform_filter1d(ele_flux_in_pass,pixel_smooth),\
                                                        uniform_filter1d(ion_flux_in_pass,pixel_smooth)

### Create Binary Input for Ion Flux (on when more than 0.1 ergs/cm^2/s/sr)

In [7]:
ion_more_ele = np.zeros_like(ion_flux_smoothed)
ion_more_ele[ion_flux_smoothed > 0.1] = 1

### Concatenate ML features

In [9]:
X = np.hstack(( ssusi_lbhl.reshape(-1,1) ,  ssusi_lbhs.reshape(-1,1), ion_more_ele.reshape(-1,1)  ))

In [10]:
y = ele_flux.reshape(-1,1) / (1.6e-12 * np.pi) # convert from erg to eV/ (cm2 * ster * S)

### Scale the Features

Use robustscaler since the data isn't quite Gaussian enough to use standard scaler

In [12]:
scaler_y = RobustScaler()
scaler_x = RobustScaler()

scaler_x.fit(X)
scaler_y.fit(y)

X = scaler_x.transform(X)
y = scaler_y.transform(y)

Save the scalers 

In [13]:
#save the scalers for later analysis
pickle.dump(scaler_x, open('X_scaler.pkl','wb'))
pickle.dump(scaler_y, open('Y_scaler.pkl','wb'))

Save all of the Model Inputs

In [14]:
h5f = h5py.File('model2_inputs.h5','w')
h5f.create_dataset('X',data = X)
h5f.create_dataset('y',data = y)
h5f.create_dataset('ele_flux',data = ele_flux)
h5f.create_dataset('ion_flux_smoothed',data = ion_flux_smoothed)
h5f.create_dataset('ele_flux_smoothed',data = ele_flux_smoothed)
h5f.create_dataset('ion_flux',data = ion_flux)
h5f.create_dataset('ssusi_lbhl', data = ssusi_lbhl)
h5f.create_dataset('ssusi_lbhs', data = ssusi_lbhs)
h5f.create_dataset('sat_nums', data = sat_nums)
h5f.create_dataset('passes', data = passes)
h5f.create_dataset('jds', data = jds)
h5f.create_dataset('lons', data = lons)
h5f.create_dataset('lats', data = lats)
h5f.create_dataset('ele_diff_energy_flux_arr', data = ele_diff_energy_flux_arr)
h5f.close()

# Neural Network Training

Really the only thing of note with this model is the usage of leakyReLu. I choice that activation function because I really wanted to enforce nonzero neuron activation and as a result enforce non linearity. 

In [16]:
def get_model(n_inputs,n_outputs):
    """
    Super Simple sequential neural network model
    """
    NN_model = keras.Sequential()

    # The Input Layer :
    NN_model.add(keras.layers.Dense(8, kernel_initializer='normal',input_dim = n_inputs, activation='relu'))
    
    # The Hidden Layers :
    hidden_layer = tf.keras.layers.LeakyReLU(32 ) 
    NN_model.add(hidden_layer)

    # The Output Layer :
    NN_model.add(keras.layers.Dense(n_outputs, kernel_initializer='normal',activation='linear') )
    
    opt = keras.optimizers.Adam(learning_rate = 0.01)
    
    # Compile the network :
    NN_model.compile(loss='mse', optimizer= opt, metrics=['mse','mean_absolute_percentage_error'])

    return NN_model

## Test Train split 

Here I wanted to kep track of which passes were test and which weren't. If that's not important ignore the hardcoding here.

In [17]:
unique_passes = np.unique(passes)
len(unique_passes)

580

In [18]:
# let 10% of passes be test passes
test_ratio = 0.1 
num_test_passes = int(test_ratio * len(unique_passes) ) - 1

#pick random passes to be test passes
index = np.random.choice(unique_passes, num_test_passes, replace=False) 
test_pass_nums = [ 12.,  18.,  22.,  35.,  51.,  70.,  72.,  83.,  88.,  91., 112.,
       113., 135., 137., 150., 158., 161., 168., 184., 189., 205., 208.,
       216., 217., 222., 228., 240., 244., 261., 263., 267., 272., 274.,
       275., 277., 285., 306., 316., 318., 323., 337., 348., 359., 372.,
       387., 395., 416., 420., 428., 436., 466., 481., 513., 520., 538.,
       555., 566.]
test_mask = np.zeros_like(passes, dtype = bool)
for pass_ind in test_pass_nums:
    test_mask = np.logical_or(test_mask, passes == pass_ind)
train_mask = np.logical_not(test_mask)

In [19]:
y_test, y_train = y[test_mask], y[train_mask]
X_test, X_train = X[test_mask], X[train_mask]
jd_test = jds[test_mask] 
ssj_lbhl_test, ssj_lbhs_test = ssj_lbhl[test_mask] , ssj_lbhs[test_mask] 
sat_nums_test = sat_nums[test_mask]
#add lat lon

## Fit the model 

In [20]:
#get training data
index = np.random.choice(np.arange(len(X_train)), int(test_ratio * len(X_train) ), replace=False)  
X_train_percent = X_train[index]
y_train_percent = y_train[index]
#train the nn
model = get_model(3,1)

#this callbackwill stop the training when there is no improvement in validation loss for 5 consecutive epochs
# callback = tf.keras.callbacks.EarlyStopping(monitor = 'loss', patience = 20,restore_best_weights = True)
callback = tf.keras.callbacks.EarlyStopping(monitor = 'loss', patience = 40,restore_best_weights = True)

model.fit(X, y, epochs=200, batch_size=32, validation_split = 0.2, callbacks = [callback])


Epoch 1/200
17304/17304 [==============================] - 10s 556us/step - loss: 46.0778 - mse: 46.0778 - mean_absolute_percentage_error: 5147.5972 - val_loss: 50.9570 - val_mse: 50.9570 - val_mean_absolute_percentage_error: 4359.6064
Epoch 2/200
17304/17304 [==============================] - 10s 567us/step - loss: 45.1658 - mse: 45.1658 - mean_absolute_percentage_error: 5120.1177 - val_loss: 52.5739 - val_mse: 52.5739 - val_mean_absolute_percentage_error: 5355.2075
Epoch 3/200
17304/17304 [==============================] - 9s 529us/step - loss: 44.9425 - mse: 44.9425 - mean_absolute_percentage_error: 4988.9722 - val_loss: 50.5599 - val_mse: 50.5599 - val_mean_absolute_percentage_error: 4835.8374
Epoch 4/200
17304/17304 [==============================] - 9s 530us/step - loss: 44.8637 - mse: 44.8637 - mean_absolute_percentage_error: 4772.6914 - val_loss: 56.7475 - val_mse: 56.7475 - val_mean_absolute_percentage_error: 3296.0205
Epoch 5/200
17304/17304 [==============================] -

### Save the model

In [21]:
model.save('lbh_ion_mask_to_ele_flux_3')

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: lbh_ion_mask_to_ele_flux_3/assets
